In [7]:
#Dependencies
import numpy as np
import pandas as pd
import json
import requests
from config import api_key
from pprint import pprint
#from pandas_profiling import ProfileReport

In [8]:
url = "https://api.covidactnow.org/v2/county/MI.json?"

# Build query URL
query_url = f"{url}apiKey={api_key}"

In [9]:
# Current data for all states
covid_19_data = requests.get(query_url).json()

pprint(covid_19_data[0])

{'actuals': {'cases': 739,
             'contactTracers': None,
             'deaths': 32,
             'hospitalBeds': {'capacity': None,
                              'currentUsageCovid': None,
                              'currentUsageTotal': None,
                              'typicalUsageRate': None},
             'icuBeds': {'capacity': None,
                         'currentUsageCovid': None,
                         'currentUsageTotal': None,
                         'typicalUsageRate': None},
             'negativeTests': None,
             'newCases': 1,
             'newDeaths': 0,
             'positiveTests': None,
             'vaccinationsCompleted': 4847,
             'vaccinationsInitiated': 5121,
             'vaccinationsInitiatedDemographics': None,
             'vaccinesAdministered': 9968,
             'vaccinesAdministeredDemographics': None,
             'vaccinesDistributed': None},
 'annotations': {'caseDensity': {'anomalies': [],
                           

In [11]:
states_list = []
fips_list = []
county_list = []
populations = []
cases = []
vaccinations_Completed = []
vaccinations_Initiated = []
vaccinations_Administered = []
for x in covid_19_data:
    states_list.append(x['state'])  
    fips_list.append(x['fips'])  
    county_list.append(x['county']) 
    populations.append(x['population'])  
    cases.append(x['actuals']['cases'])
    vaccinations_Completed.append(x['actuals']['vaccinationsCompleted'])
    vaccinations_Initiated.append(x['actuals']['vaccinationsInitiated']) 
    vaccinations_Administered.append(x['actuals']['vaccinesAdministered'])

In [22]:
# create dataframe
covid_19_df = pd.DataFrame({
    "State": states_list,
    "Fips" : fips_list,
    "County" : county_list,
    "Population": populations,
    "Total case (Current)" : cases,
    "Vaccination Completed" : vaccinations_Completed,
    "Vaccination Initiated" : vaccinations_Initiated,
    "Vaccinations Administered":vaccinations_Administered
})
covid_19_df.head()

,State,Fips,County,Population,Total case (Current),Vaccination Completed,Vaccination Initiated,Vaccinations Administered
0,MI,26001,Alcona County,10405,739,4847,5121,9968
1,MI,26003,Alger County,9108,667,4418,4915,9333
2,MI,26005,Allegan County,118081,11134,44494,49801,94295
3,MI,26007,Alpena County,28405,2263,12436,13226,25662
4,MI,26009,Antrim County,23324,1629,10807,11622,22429


In [23]:
covid_19_df["Percent Completed"]=covid_19_df["Vaccination Completed"]/covid_19_df["Population"]*100

In [24]:
covid_19_df.head()

,State,Fips,County,Population,Total case (Current),Vaccination Completed,Vaccination Initiated,Vaccinations Administered,Percent Completed
0,MI,26001,Alcona County,10405,739,4847,5121,9968,46.583373
1,MI,26003,Alger County,9108,667,4418,4915,9333,48.506807
2,MI,26005,Allegan County,118081,11134,44494,49801,94295,37.680914
3,MI,26007,Alpena County,28405,2263,12436,13226,25662,43.781024
4,MI,26009,Antrim County,23324,1629,10807,11622,22429,46.334248


In [25]:

covid_19_df.agg({'Percent Completed': ['min', 'max']})

,Percent Completed
min,23.779631
max,60.305133


In [28]:

grouped = covid_19_df.groupby(["County", "Percent Completed"])
grouped.head()

,State,Fips,County,Population,Total case (Current),Vaccination Completed,Vaccination Initiated,Vaccinations Administered,Percent Completed
0,MI,26001,Alcona County,10405,739,4847,5121,9968,46.583373
1,MI,26003,Alger County,9108,667,4418,4915,9333,48.506807
2,MI,26005,Allegan County,118081,11134,44494,49801,94295,37.680914
3,MI,26007,Alpena County,28405,2263,12436,13226,25662,43.781024
4,MI,26009,Antrim County,23324,1629,10807,11622,22429,46.334248
...,...,...,...,...,...,...,...,...,...
78,MI,26157,Tuscola County,52245,5662,18368,20242,38610,35.157431
79,MI,26159,Van Buren County,75677,7118,29784,33661,63445,39.356740
80,MI,26161,Washtenaw County,367601,27062,187594,207797,395391,51.031961
81,MI,26163,Wayne County,1749343,164994,637971,756151,1394122,36.469177
